In [ ]:
# Bag of Words Sentiment Analyse von McDonalds Reviews

## Zielsetzung 
In diesem Notebook teilen wir die Testdaten in positven und Negative Reviews auf. Dadurch haben wir nur zwei Labels. Als common sense Baseline dient dementsprechend 50% accuracy.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers


## Datenvorbereitung inkl. Behandlung bekannter Probleme

### Laden der Daten

In [1]:
%pip install kaggle
!kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
!unzip mcdonalds-store-reviews.zip

!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
mcdonalds-store-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  mcdonalds-store-reviews.zip
replace McDonald_s_Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 